In [12]:
import vertexai
# from vertexai.generative_models import GenerativeModel
from vertexai.generative_models import GenerativeModel, ChatSession, Part, Content
import vertexai.preview.generative_models as generative_models

In [13]:
import os
from dotenv import load_dotenv

load_dotenv()


True

In [14]:
import os
vertexai.init(project="vision-forge-414908", location="us-central1")
os.environ['GOOGLE_API_KEY']=os.getenv("GOOGLE_API_KEY")
os.environ['GOOGLE_APPLICATION_CREDENTIALS']="cred.json"

In [15]:
model = GenerativeModel(model_name="gemini-1.5-flash-001",
                             system_instruction="You have to have a conversation with the user about their insurance-related experience. This may be a hospitalization/car accident etc. Ask relevant questions, ask for necessary clarifications, etc. Once you are done with understanding their experience, the user will send you a list of necessary fields they require to fill out a form. You must return a json object with the values for these fields. Ensure that all your information is accurate, elaborate, and professional. This json object will be your final output.",
                            
     )

In [16]:
user_profile = {
  "patient_profile": {
    "personal_details": {
      "name": "Ranjit Sharma",
      "date_of_birth": "1985-06-15",
      "gender": "Male",
      "marital_status": "Married",
      "children": 0,
      "spouse_name": "Anita Sharma",
      "contact_details": {
        "phone": "+91-9876543210",
        "email": "ranjit.sharma@example.com",
        "address": {
          "street": "15, MG Road",
          "city": "Bengaluru",
          "state": "Karnataka",
          "postal_code": "560001",
          "country": "India"
        }
      }
    },
    "insurance_details": {
      "policy_number": "HSI123456789",
      "policy_provider": "SBI Health Insurance",
      "policy_path": "../docs/SBI.pdf",
      "policy_start_date": "2020-01-01",
      "policy_end_date": "2025-01-01",
      "coverage_amount": 500000,
      "premium_amount": 12000,
      "premium_frequency": "Yearly"
    },
    "medical_history": {
      "pre_existing_conditions": [],
      "allergies": [],
      "current_medications": [],
      "past_treatments": []
    },
    "emergency_contact": {
      "name": "Anita Sharma",
      "relationship": "Spouse",
      "phone": "+91-9876543211"
    },
    "claim_history": [
      {
        "claim_id": "CLAIM98765",
        "claim_date": "2023-04-15",
        "hospital_name": "Apollo Hospital",
        "treatment": "Fracture Treatment",
        "claim_amount": 50000,
        "status": "Approved"
      }
    ]
  }
}

In [17]:
chat_history = [
    {
        "role": "user",
        "text": "This is my user profile:\n"+ str(user_profile)
    },
    {
        "role": "model",
        "text": "Okay, got it. Please ask your questions related to insurance."
    },
    
]

In [18]:
history = []
for message in chat_history:
    if message["role"] == "user":
        history.append(
            Content(role="user", parts=[Part.from_text(message["text"])])
        )
    else:
        history.append(
            Content(role="model", parts=[Part.from_text(f'{message["text"]}')])
        )

In [19]:
chat = ChatSession(model=model, history=history)

In [24]:
op = chat.send_message("She was admitted in an in network hospital called Manipal for 3 days from 28/3/2024 - 31/3/2024. She has diabetes. We had a total of 32 thousand rupees in doctor fees. ")

In [25]:
op.candidates[0].content.parts[0].text

"Thank you for providing that information. It's helpful to know about your wife's diabetes. It's likely that this is a pre-existing condition that will need to be considered when processing your claim.  \n\nTo give you a clear idea of your options, I'd need a bit more information. Could you please tell me:\n\n* **What type of insurance policy do you have?** Is it a health insurance policy, a hospitalization policy, or something else?\n* **Does your policy specifically cover dengue?** \n* **What is your policy's maximum coverage limit for a single hospitalization?** \n\nOnce I have this information, I can assess whether your claim is likely to be approved and what documentation you'll need to gather. \n"

In [37]:
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoModel
model_name = "jinaai/jina-embeddings-v2-base-en"
model_kwargs ={'trust_remote_code': True}


In [39]:
embeddings_model = HuggingFaceEmbeddings(model_name="jinaai/jina-embeddings-v2-small-en",model_kwargs=model_kwargs)
